In [12]:
from BSE_B import market_session # version 1.9.1 
from datetime import datetime
import csv
import config
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [5]:
def schedule_offsetfn_read_file(filename, col_t, col_p, scale_factor=75):
       
        # does two passes through the file
        # assumes data file is all for one date, sorted in time order, in correct format, etc. etc.
        rwd_csv = csv.reader(open(filename, 'r'))
        # first pass: get time & price events, find out how long session is, get min & max price
        minprice = None
        maxprice = None
        firsttimeobj = None
        timesincestart = 0
        priceevents = []
        for line in rwd_csv:
            print(line)
            time = line[col_t]
            if firsttimeobj is None:
                firsttimeobj = datetime.strptime(time, '%H:%M:%S')
            timeobj = datetime.strptime(time, '%H:%M:%S')
            price = float(line[col_p])
            if minprice is None or price < minprice:
                minprice = price
            if maxprice is None or price > maxprice:
                maxprice = price
            timesincestart = (timeobj - firsttimeobj).total_seconds()
            priceevents.append([timesincestart, price])
        # second pass: normalise times to fractions of entire time-series duration
        #              & normalise price range
        pricerange = maxprice - minprice
        endtime = float(timesincestart)
        offsetfn_eventlist = []
        for event in priceevents:
            # normalise price
            normld_price = (event[1] - minprice) / pricerange
            # clip
            normld_price = min(normld_price, 1.0)
            normld_price = max(0.0, normld_price)
            # scale & convert to integer cents
            price = int(round(normld_price * scale_factor))
            normld_event = [event[0] / endtime, price]
            offsetfn_eventlist.append(normld_event)
        return offsetfn_eventlist

In [8]:
def schedule_offsetfn_from_eventlist(time, params):
    final_time = float(params[0])
    offset_events = params[1]
    # this is quite inefficient: on every call it walks the event-list
    percent_elapsed = time/final_time
    offset = None
    for event in offset_events:
        offset = event[1]
        if percent_elapsed < event[0]:
            break
    return offset

## No Market Shock

In [477]:
import importlib
import BSEv1_91_ALTR
importlib.reload(BSEv1_91_ALTR)
from BSEv1_91_ALTR import market_session

In [479]:
start_time = 0.0
end_time = 1000

In [485]:
# 1000 days is often good, but 3*365=1095, so may as well go for three years.
n_days = 1
hours_in_a_day = 7.5
start_time = 0.0
end_time = 60.0 * 60.0 * hours_in_a_day * n_days
duration = end_time - start_time

In [396]:
# offset
price_offset_filename = 'offset-ibm-1m-170831.csv'
offsetfn_events = schedule_offsetfn_read_file(price_offset_filename, 1, 2)

['ï»¿31/08/2017', '10:31:00', '143.56', '143.61', '143.56', '143.58', '3958']
['31/08/2017', '10:32:00', '143.56', '143.6', '143.5401', '143.59', '19667']
['31/08/2017', '10:33:00', '143.58', '143.59', '143.491', '143.53', '6412']
['31/08/2017', '10:34:00', '143.54', '143.54', '143.48', '143.4938', '7009']
['31/08/2017', '10:35:00', '143.49', '143.5', '143.47', '143.48', '4202']
['31/08/2017', '10:36:00', '143.47', '143.4733', '143.44', '143.46', '5408']
['31/08/2017', '10:37:00', '143.46', '143.46', '143.26', '143.28', '16384']
['31/08/2017', '10:38:00', '143.27', '143.3', '143.19', '143.3', '9497']
['31/08/2017', '10:39:00', '143.31', '143.38', '143.28', '143.379', '4875']
['31/08/2017', '10:40:00', '143.39', '143.39', '143.32', '143.34', '4898']
['31/08/2017', '10:41:00', '143.37', '143.37', '143.32', '143.34', '4871']
['31/08/2017', '10:42:00', '143.355', '143.39', '143.355', '143.385', '1763']
['31/08/2017', '10:43:00', '143.41', '143.41', '143.32', '143.32', '7000']
['31/08/2017'

In [453]:
# offset
price_offset_filename = 'offset-nasdaq-1m-170913.csv'
offsetfn_events = schedule_offsetfn_read_file(price_offset_filename, 1, 2)

['ï»¿13/09/2017', '00:46:00', '5999.75', '5999.75', '5999.5', '5999.5', '10']
['13/09/2017', '00:47:00', '5999.75', '5999.75', '5999.5', '5999.5', '3']
['13/09/2017', '00:48:00', '5999.75', '5999.75', '5999.75', '5999.75', '4']
['13/09/2017', '00:49:00', '5999.75', '6000', '5999.75', '6000', '10']
['13/09/2017', '00:51:00', '5999.75', '5999.75', '5999.75', '5999.75', '2']
['13/09/2017', '00:52:00', '5999.75', '5999.75', '5999.5', '5999.5', '5']
['13/09/2017', '00:53:00', '5999.25', '5999.25', '5998.75', '5999.25', '35']
['13/09/2017', '00:54:00', '5999.25', '5999.25', '5999.25', '5999.25', '5']
['13/09/2017', '00:55:00', '5999.25', '5999.25', '5998.25', '5998.25', '21']
['13/09/2017', '00:56:00', '5998.5', '5999', '5998.25', '5998.25', '56']
['13/09/2017', '00:57:00', '5998.25', '5998.5', '5998', '5998', '48']
['13/09/2017', '00:58:00', '5997.75', '5998', '5997.25', '5997.5', '59']
['13/09/2017', '00:59:00', '5997.5', '5997.75', '5997.5', '5997.75', '10']
['13/09/2017', '01:00:00', '59

In [481]:
## MMM01
range1 = (50, 100, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [range1], 'stepmode': 'jittered'}]

range2 = (150, 200, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [range2], 'stepmode': 'jittered'}]

# new customer orders arrive at each trader approx once every order_interval seconds
order_interval = 20

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                'interval': order_interval, 'timemode': 'drip-poisson'}

# run a sequence of trials, one session per trial

verbose = config.verbose

# n_trials is how many trials (i.e. market sessions) to run in total
n_trials = 3

# n_recorded is how many trials (i.e. market sessions) to write full data-files for
n_trials_recorded = 3

trial = 1

while trial < (n_trials+1):

    # create unique i.d. string for this trial
    trial_id = 'MMM01_d%03d_i%02d_%04d' % (n_days, order_interval, trial)

    buyers_spec = [('SHVR', 1), ('GVWY', 1), ('ZIC', 3), ('ZIP', 6), ('SNPR', 1),
                   ('ZIPSH', 3, {'k':5}), ('PRZI', 3, {'s_min': -1.0, 's_max': +1.0}),
                   ('PRSH', 2, {'s_min': -1.0, 's_max': +1.0, 'k':5})]
    sellers_spec = buyers_spec
    mrktmakers_spec = [('MMM01', 1)]

    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec, 'mrktmakers': mrktmakers_spec}

    if trial > n_trials_recorded:
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                        'dump_avgbals': False, 'dump_tape': False}
    else:
        dump_flags = {'dump_blotters': True, 'dump_lobs': False, 'dump_strats': True,
                        'dump_avgbals': True, 'dump_tape': True}

    market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

    trial = trial + 1

B15: PRZI active_strat=[0]:
[0]: s=+0.266383, start=0.000000, $=0.000000, pps=0.000000


B16: PRZI active_strat=[0]:
[0]: s=+0.500401, start=0.000000, $=0.000000, pps=0.000000


B17: PRZI active_strat=[0]:
[0]: s=-0.685299, start=0.000000, $=0.000000, pps=0.000000


B18: PRSH active_strat=[0]:
[0]: s=+0.978622, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.936740, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.957628, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.981765, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+1.000000, start=0.000000, $=0.000000, pps=0.000000


B19: PRSH active_strat=[0]:
[0]: s=+0.243260, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.359024, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.211174, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.263242, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+0.130474, start=0.000000, $=0.000000, pps=0.000000


S15: PRZI active_strat=[0]:
[0]: s=+0.557560, start=0.000000, $=0.000000, pps=0.000

In [487]:
## MMM02
range1 = (50, 100, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [range1], 'stepmode': 'jittered'}]

range2 = (150, 200, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [range2], 'stepmode': 'jittered'}]

# new customer orders arrive at each trader approx once every order_interval seconds
order_interval = 20

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                'interval': order_interval, 'timemode': 'drip-poisson'}

# run a sequence of trials, one session per trial

verbose = config.verbose

# n_trials is how many trials (i.e. market sessions) to run in total
n_trials = 2

# n_recorded is how many trials (i.e. market sessions) to write full data-files for
n_trials_recorded = 2

trial = 1

while trial < (n_trials+1):

    # create unique i.d. string for this trial
    trial_id = 'MMM02_d%03d_i%02d_%04d' % (n_days, order_interval, trial)

    buyers_spec = [('SHVR', 1), ('GVWY', 1), ('ZIC', 3), ('ZIP', 6), ('SNPR', 1),
                   ('ZIPSH', 3, {'k':5}), ('PRZI', 3, {'s_min': -1.0, 's_max': +1.0}),
                   ('PRSH', 2, {'s_min': -1.0, 's_max': +1.0, 'k':5})]
    sellers_spec = buyers_spec
    mrktmakers_spec = [('MMM02', 1, {'n_past_trades_short': 5,
                                     'n_past_trades_long': 10,
                                     'ask_delta': 15})]

    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec, 'mrktmakers': mrktmakers_spec}

    if trial > n_trials_recorded:
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                        'dump_avgbals': False, 'dump_tape': False}
    else:
        dump_flags = {'dump_blotters': True, 'dump_lobs': False, 'dump_strats': True,
                        'dump_avgbals': True, 'dump_tape': True}

    market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

    trial = trial + 1

B15: PRZI active_strat=[0]:
[0]: s=-0.505614, start=0.000000, $=0.000000, pps=0.000000


B16: PRZI active_strat=[0]:
[0]: s=-0.499699, start=0.000000, $=0.000000, pps=0.000000


B17: PRZI active_strat=[0]:
[0]: s=+0.928519, start=0.000000, $=0.000000, pps=0.000000


B18: PRSH active_strat=[0]:
[0]: s=+0.401489, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.453475, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.450908, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.510092, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+0.434854, start=0.000000, $=0.000000, pps=0.000000


B19: PRSH active_strat=[0]:
[0]: s=-0.077929, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.063475, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.125867, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.030312, start=0.000000, $=0.000000, pps=0.000000
[4]: s=-0.047373, start=0.000000, $=0.000000, pps=0.000000


S15: PRZI active_strat=[0]:
[0]: s=+0.890586, start=0.000000, $=0.000000, pps=0.000

KeyboardInterrupt: 

## With Positive Market Shock

In [460]:
# offset
price_offset_filename = 'offset-ibm-1m-170831.csv'
offsetfn_events = schedule_offsetfn_read_file(price_offset_filename, 1, 2)

['ï»¿31/08/2017', '10:31:00', '143.56', '143.61', '143.56', '143.58', '3958']
['31/08/2017', '10:32:00', '143.56', '143.6', '143.5401', '143.59', '19667']
['31/08/2017', '10:33:00', '143.58', '143.59', '143.491', '143.53', '6412']
['31/08/2017', '10:34:00', '143.54', '143.54', '143.48', '143.4938', '7009']
['31/08/2017', '10:35:00', '143.49', '143.5', '143.47', '143.48', '4202']
['31/08/2017', '10:36:00', '143.47', '143.4733', '143.44', '143.46', '5408']
['31/08/2017', '10:37:00', '143.46', '143.46', '143.26', '143.28', '16384']
['31/08/2017', '10:38:00', '143.27', '143.3', '143.19', '143.3', '9497']
['31/08/2017', '10:39:00', '143.31', '143.38', '143.28', '143.379', '4875']
['31/08/2017', '10:40:00', '143.39', '143.39', '143.32', '143.34', '4898']
['31/08/2017', '10:41:00', '143.37', '143.37', '143.32', '143.34', '4871']
['31/08/2017', '10:42:00', '143.355', '143.39', '143.355', '143.385', '1763']
['31/08/2017', '10:43:00', '143.41', '143.41', '143.32', '143.32', '7000']
['31/08/2017'

In [462]:
# time
n_days = 1
hours_in_a_day = 7.5

start_time = 0.0
end_time = 60.0 * 60.0 * hours_in_a_day * n_days
duration = end_time - start_time

# time of market shock
mid_time = end_time/2

In [497]:
duration

27000.0

In [469]:
## MMM01
# demand supply curves
range1 = (50, 100, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range1_pos = (100, 150, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
supply_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range1], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range1_pos], 'stepmode': 'jittered'}]

range2 = (150, 200, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range2_pos = (200, 250, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
demand_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range2], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range2_pos], 'stepmode': 'jittered'}]


# new customer orders arrive at each trader approx once every order_interval seconds
order_interval = 20

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                'interval': order_interval, 'timemode': 'drip-poisson'}

# run a sequence of trials, one session per trial
verbose = config.verbose

# n_trials is how many trials (i.e. market sessions) to run in total
n_trials = 2

# n_recorded is how many trials (i.e. market sessions) to write full data-files for
n_trials_recorded = 2

trial = 1

while trial < (n_trials+1):

    # create unique i.d. string for this trial
    trial_id = 'MMM01_pos_ibm_d%03d_i%02d_%04d' % (n_days, order_interval, trial)

    buyers_spec = [('SHVR', 1), ('GVWY', 1), ('ZIC', 3), ('ZIP', 6), ('SNPR', 1),
                   ('ZIPSH', 3, {'k':5}), ('PRZI', 3, {'s_min': -1.0, 's_max': +1.0}),
                   ('PRSH', 2, {'s_min': -1.0, 's_max': +1.0, 'k':5})]
    sellers_spec = buyers_spec
    mrktmakers_spec = [('MMM01', 1, {'n_past_trades_short': 5,
                                     'n_past_trades_long': 10,
                                     'ask_delta': 15})]

    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec, 'mrktmakers': mrktmakers_spec}

    if trial > n_trials_recorded:
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                        'dump_avgbals': False, 'dump_tape': False}
    else:
        dump_flags = {'dump_blotters': True, 'dump_lobs': False, 'dump_strats': True,
                        'dump_avgbals': True, 'dump_tape': True}

    market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

    trial = trial + 1

B15: PRZI active_strat=[0]:
[0]: s=+0.345618, start=0.000000, $=0.000000, pps=0.000000


B16: PRZI active_strat=[0]:
[0]: s=+0.781658, start=0.000000, $=0.000000, pps=0.000000


B17: PRZI active_strat=[0]:
[0]: s=+0.260922, start=0.000000, $=0.000000, pps=0.000000


B18: PRSH active_strat=[0]:
[0]: s=+0.185470, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.155726, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.159351, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.165654, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+0.141983, start=0.000000, $=0.000000, pps=0.000000


B19: PRSH active_strat=[0]:
[0]: s=+0.568951, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.471722, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.575404, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.575927, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+0.579103, start=0.000000, $=0.000000, pps=0.000000


S15: PRZI active_strat=[0]:
[0]: s=+0.618626, start=0.000000, $=0.000000, pps=0.000

In [390]:
## MMM02
# demand supply curves
range1 = (50, 100, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range1_pos = (100, 150, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
supply_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range1], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range1_pos], 'stepmode': 'jittered'}]

range2 = (150, 200, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range2_pos = (200, 250, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
demand_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range2], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range2_pos], 'stepmode': 'jittered'}]


# new customer orders arrive at each trader approx once every order_interval seconds
order_interval = 20

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                'interval': order_interval, 'timemode': 'drip-poisson'}

# run a sequence of trials, one session per trial
verbose = config.verbose

# n_trials is how many trials (i.e. market sessions) to run in total
n_trials = 2

# n_recorded is how many trials (i.e. market sessions) to write full data-files for
n_trials_recorded = 2

trial = 1

while trial < (n_trials+1):

    # create unique i.d. string for this trial
    trial_id = 'MMM02_pos_ibm_d%03d_i%02d_%04d' % (n_days, order_interval, trial)

    buyers_spec = [('SHVR', 1), ('GVWY', 1), ('ZIC', 3), ('ZIP', 6), ('SNPR', 1),
                   ('ZIPSH', 3, {'k':5}), ('PRZI', 3, {'s_min': -1.0, 's_max': +1.0}),
                   ('PRSH', 2, {'s_min': -1.0, 's_max': +1.0, 'k':5})]
    sellers_spec = buyers_spec
    mrktmakers_spec = [('MMM02', 1, {'n_past_trades_short': 5,
                                     'n_past_trades_long': 10,
                                     'ask_delta': 15})]

    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec, 'mrktmakers': mrktmakers_spec}

    if trial > n_trials_recorded:
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                        'dump_avgbals': False, 'dump_tape': False}
    else:
        dump_flags = {'dump_blotters': True, 'dump_lobs': False, 'dump_strats': True,
                        'dump_avgbals': True, 'dump_tape': True}

    market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

    trial = trial + 1

B15: PRZI active_strat=[0]:
[0]: s=-0.506320, start=0.000000, $=0.000000, pps=0.000000


B16: PRZI active_strat=[0]:
[0]: s=-0.248041, start=0.000000, $=0.000000, pps=0.000000


B17: PRZI active_strat=[0]:
[0]: s=-0.208668, start=0.000000, $=0.000000, pps=0.000000


B18: PRSH active_strat=[0]:
[0]: s=+0.960354, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.986669, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.980425, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+1.000000, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+1.000000, start=0.000000, $=0.000000, pps=0.000000


B19: PRSH active_strat=[0]:
[0]: s=+0.668118, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.700143, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.717375, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.733988, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+0.675656, start=0.000000, $=0.000000, pps=0.000000


S15: PRZI active_strat=[0]:
[0]: s=+0.950804, start=0.000000, $=0.000000, pps=0.000

## Negatie Market Shock

In [493]:
# offset
price_offset_filename = 'offset-ibm-1m-170831.csv'
offsetfn_events = schedule_offsetfn_read_file(price_offset_filename, 1, 2)

['ï»¿31/08/2017', '10:31:00', '143.56', '143.61', '143.56', '143.58', '3958']
['31/08/2017', '10:32:00', '143.56', '143.6', '143.5401', '143.59', '19667']
['31/08/2017', '10:33:00', '143.58', '143.59', '143.491', '143.53', '6412']
['31/08/2017', '10:34:00', '143.54', '143.54', '143.48', '143.4938', '7009']
['31/08/2017', '10:35:00', '143.49', '143.5', '143.47', '143.48', '4202']
['31/08/2017', '10:36:00', '143.47', '143.4733', '143.44', '143.46', '5408']
['31/08/2017', '10:37:00', '143.46', '143.46', '143.26', '143.28', '16384']
['31/08/2017', '10:38:00', '143.27', '143.3', '143.19', '143.3', '9497']
['31/08/2017', '10:39:00', '143.31', '143.38', '143.28', '143.379', '4875']
['31/08/2017', '10:40:00', '143.39', '143.39', '143.32', '143.34', '4898']
['31/08/2017', '10:41:00', '143.37', '143.37', '143.32', '143.34', '4871']
['31/08/2017', '10:42:00', '143.355', '143.39', '143.355', '143.385', '1763']
['31/08/2017', '10:43:00', '143.41', '143.41', '143.32', '143.32', '7000']
['31/08/2017'

In [491]:
# time
n_days = 1
hours_in_a_day = 7.5

start_time = 0.0
end_time = 60.0 * 60.0 * hours_in_a_day * n_days
duration = end_time - start_time

# time of market shock
mid_time = end_time/2

In [495]:
## MMM01
# demand supply curves
range1 = (50, 100, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range1_neg = (10, 50, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
supply_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range1], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range1_neg], 'stepmode': 'jittered'}]

range2 = (150, 200, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range2_neg = (100, 150, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
demand_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range2], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range2_neg], 'stepmode': 'jittered'}]


# new customer orders arrive at each trader approx once every order_interval seconds
order_interval = 20

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                'interval': order_interval, 'timemode': 'drip-poisson'}

# run a sequence of trials, one session per trial
verbose = config.verbose

# n_trials is how many trials (i.e. market sessions) to run in total
n_trials = 5

# n_recorded is how many trials (i.e. market sessions) to write full data-files for
n_trials_recorded = 5

trial = 1

while trial < (n_trials+1):

    # create unique i.d. string for this trial
    trial_id = 'MMM01_neg_ibm_d%03d_i%02d_%04d' % (n_days, order_interval, trial)

    buyers_spec = [('SHVR', 1), ('GVWY', 1), ('ZIC', 3), ('ZIP', 6), ('SNPR', 1),
                   ('ZIPSH', 3, {'k':5}), ('PRZI', 3, {'s_min': -1.0, 's_max': +1.0}),
                   ('PRSH', 2, {'s_min': -1.0, 's_max': +1.0, 'k':5})]
    sellers_spec = buyers_spec
    mrktmakers_spec = [('MMM01', 1, {'n_past_trades_short': 5,
                                     'n_past_trades_long': 10,
                                     'ask_delta': 15})]

    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec, 'mrktmakers': mrktmakers_spec}

    if trial > n_trials_recorded:
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                        'dump_avgbals': False, 'dump_tape': False}
    else:
        dump_flags = {'dump_blotters': True, 'dump_lobs': False, 'dump_strats': True,
                        'dump_avgbals': True, 'dump_tape': True}

    market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

    trial = trial + 1

B15: PRZI active_strat=[0]:
[0]: s=-0.306379, start=0.000000, $=0.000000, pps=0.000000


B16: PRZI active_strat=[0]:
[0]: s=-0.142892, start=0.000000, $=0.000000, pps=0.000000


B17: PRZI active_strat=[0]:
[0]: s=+0.041262, start=0.000000, $=0.000000, pps=0.000000


B18: PRSH active_strat=[0]:
[0]: s=-0.908165, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.913250, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.988789, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.936026, start=0.000000, $=0.000000, pps=0.000000
[4]: s=-0.978387, start=0.000000, $=0.000000, pps=0.000000


B19: PRSH active_strat=[0]:
[0]: s=+0.747717, start=0.000000, $=0.000000, pps=0.000000
[1]: s=+0.710536, start=0.000000, $=0.000000, pps=0.000000
[2]: s=+0.701651, start=0.000000, $=0.000000, pps=0.000000
[3]: s=+0.706742, start=0.000000, $=0.000000, pps=0.000000
[4]: s=+0.690700, start=0.000000, $=0.000000, pps=0.000000


S15: PRZI active_strat=[0]:
[0]: s=-0.457874, start=0.000000, $=0.000000, pps=0.000

In [417]:
## MMM02
# demand supply curves
range1 = (50, 100, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range1_neg = (10, 50, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
supply_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range1], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range1_neg], 'stepmode': 'jittered'}]

range2 = (150, 200, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
range2_neg = (100, 150, (schedule_offsetfn_from_eventlist, [[end_time, offsetfn_events]]))
demand_schedule = [{'from': start_time, 'to': mid_time, 'ranges': [range2], 'stepmode': 'jittered'},
                   {'from': mid_time, 'to': end_time, 'ranges': [range2_neg], 'stepmode': 'jittered'}]


# new customer orders arrive at each trader approx once every order_interval seconds
order_interval = 20

order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
                'interval': order_interval, 'timemode': 'drip-poisson'}

# run a sequence of trials, one session per trial
verbose = config.verbose

# n_trials is how many trials (i.e. market sessions) to run in total
n_trials = 1

# n_recorded is how many trials (i.e. market sessions) to write full data-files for
n_trials_recorded = 1

trial = 1

while trial < (n_trials+1):

    # create unique i.d. string for this trial
    trial_id = 'MMM02_neg_ibm_d%03d_i%02d_%04d' % (n_days, order_interval, trial)

    buyers_spec = [('SHVR', 1), ('GVWY', 1), ('ZIC', 3), ('ZIP', 6), ('SNPR', 1),
                   ('ZIPSH', 3, {'k':5}), ('PRZI', 3, {'s_min': -1.0, 's_max': +1.0}),
                   ('PRSH', 2, {'s_min': -1.0, 's_max': +1.0, 'k':5})]
    sellers_spec = buyers_spec
    mrktmakers_spec = [('MMM02', 1, {'n_past_trades_short': 5,
                                     'n_past_trades_long': 10,
                                     'ask_delta': 15})]

    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec, 'mrktmakers': mrktmakers_spec}

    if trial > n_trials_recorded:
        dump_flags = {'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False,
                        'dump_avgbals': False, 'dump_tape': False}
    else:
        dump_flags = {'dump_blotters': True, 'dump_lobs': False, 'dump_strats': True,
                        'dump_avgbals': True, 'dump_tape': True}

    market_session(trial_id, start_time, end_time, traders_spec, order_sched, dump_flags, verbose)

    trial = trial + 1

B15: PRZI active_strat=[0]:
[0]: s=-0.150941, start=0.000000, $=0.000000, pps=0.000000


B16: PRZI active_strat=[0]:
[0]: s=+0.932145, start=0.000000, $=0.000000, pps=0.000000


B17: PRZI active_strat=[0]:
[0]: s=-0.658036, start=0.000000, $=0.000000, pps=0.000000


B18: PRSH active_strat=[0]:
[0]: s=-0.496932, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.515493, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.412591, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.519141, start=0.000000, $=0.000000, pps=0.000000
[4]: s=-0.443168, start=0.000000, $=0.000000, pps=0.000000


B19: PRSH active_strat=[0]:
[0]: s=-0.472411, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.467027, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.561962, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.429265, start=0.000000, $=0.000000, pps=0.000000
[4]: s=-0.489662, start=0.000000, $=0.000000, pps=0.000000


S15: PRZI active_strat=[0]:
[0]: s=+0.378243, start=0.000000, $=0.000000, pps=0.000